In [5]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

In [3]:
daf = pd.read_csv("C:/Users/osave/Downloads/HW1_P5_weatherHistory.csv") #Importing the data
df = pd.DataFrame(daf,columns=["Temperature (C)","Apparent Temperature (C)","Humidity","Visibility (km)","Pressure (millibars)"])
df.head()

,Temperature (C),Apparent Temperature (C),Humidity,Visibility (km),Pressure (millibars)
0,9.472222,7.388889,0.89,15.8263,1015.13
1,9.355556,7.227778,0.86,15.8263,1015.63
2,9.377778,9.377778,0.89,14.9569,1015.94
3,8.288889,5.944444,0.83,15.8263,1016.41
4,8.755556,6.977778,0.83,15.8263,1016.51


In [4]:
df = df.drop(labels = "Pressure (millibars)",axis=1)
df.head()

,Temperature (C),Apparent Temperature (C),Humidity,Visibility (km)
0,9.472222,7.388889,0.89,15.8263
1,9.355556,7.227778,0.86,15.8263
2,9.377778,9.377778,0.89,14.9569
3,8.288889,5.944444,0.83,15.8263
4,8.755556,6.977778,0.83,15.8263


In [11]:
class net(nn.Module):
    def __init__(self):
        super().__init__()
        self.sequential = nn.Sequential(nn.Linear(2,10),
                                        nn.ReLU(),
                                        nn.Linear(10,1),
                                        nn.Tanh(),
                                        nn.Linear(1,1))

    def forward(self,x):
        x = self.sequential(x)
        return x

In [12]:
model = net()
model

net(
  (sequential): Sequential(
    (0): Linear(in_features=2, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=1, bias=True)
    (3): Tanh()
    (4): Linear(in_features=1, out_features=1, bias=True)
  )
)